# PYTORCH CNN Classifier

In [1]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [2]:
# Parameters
DATASET = "human_enhancers_cohn"
TOKENIZER = "character"


In [3]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

human_enhancers_cohn 0 character 0


## Config

In [4]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [5]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 8
{'<eos>': 6, 'G': 4, 'A': 5, 'C': 2, '<pad>': 7, '<bos>': 1, 'T': 3, '<unk>': 0}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device


max_tok_len  502


In [9]:
tokenizer(train_dset[1][0])

['<bos>',
 'C',
 'C',
 'A',
 'T',
 'C',
 'T',
 'A',
 'T',
 'T',
 'T',
 'T',
 'T',
 'G',
 'A',
 'A',
 'T',
 'C',
 'C',
 'T',
 'T',
 'T',
 'T',
 'A',
 'A',
 'C',
 'A',
 'C',
 'T',
 'T',
 'T',
 'A',
 'T',
 'G',
 'A',
 'G',
 'A',
 'G',
 'A',
 'G',
 'A',
 'A',
 'A',
 'G',
 'A',
 'A',
 'C',
 'A',
 'G',
 'A',
 'A',
 'C',
 'A',
 'G',
 'G',
 'C',
 'A',
 'G',
 'G',
 'C',
 'A',
 'G',
 'G',
 'C',
 'T',
 'G',
 'T',
 'G',
 'T',
 'T',
 'T',
 'A',
 'A',
 'C',
 'C',
 'C',
 'C',
 'A',
 'C',
 'A',
 'A',
 'C',
 'A',
 'C',
 'C',
 'A',
 'C',
 'C',
 'A',
 'T',
 'A',
 'G',
 'A',
 'G',
 'C',
 'C',
 'T',
 'A',
 'A',
 'A',
 'A',
 'G',
 'C',
 'T',
 'T',
 'G',
 'G',
 'G',
 'A',
 'G',
 'T',
 'A',
 'A',
 'A',
 'T',
 'A',
 'C',
 'A',
 'A',
 'T',
 'T',
 'T',
 'C',
 'T',
 'T',
 'T',
 'G',
 'G',
 'C',
 'A',
 'C',
 'C',
 'T',
 'T',
 'G',
 'T',
 'A',
 'A',
 'T',
 'A',
 'A',
 'A',
 'A',
 'C',
 'A',
 'T',
 'G',
 'A',
 'A',
 'C',
 'A',
 'A',
 'T',
 'G',
 'A',
 'G',
 'T',
 'T',
 'T',
 'C',
 'T',
 'C',
 'A',
 'C',
 'C',
 'A',


## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 64.0%, Avg loss: 0.655092 

Epoch 1


Train metrics: 
 Accuracy: 64.2%, Avg loss: 0.652506 

Epoch 2


Train metrics: 
 Accuracy: 65.5%, Avg loss: 0.650462 

Epoch 3


Train metrics: 
 Accuracy: 66.5%, Avg loss: 0.645009 

Epoch 4


Train metrics: 
 Accuracy: 67.7%, Avg loss: 0.638954 



## Testing

In [12]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  3474 ; tp  1515.9505222439766 ; fp  480.3436289012551
recall  0.4363703287979207 ; precision  0.7593823392080309
num_batches 218
correct 4572
size 6948
Test metrics: 
 Accuracy: 0.658031, F1 score: 0.554248, Avg loss: 0.646865 



(0.6580310880829016, 0.5542482653977958)